In [1]:
import numpy as np
from numpy import genfromtxt
from collections import defaultdict
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import csv
from numpy import genfromtxt
from tensorflow.keras.models import Model

##  Prepare Data

You can find the dataset on [MovieLens ml-latest-small](https://grouplens.org/datasets/movielens/latest/). 
* The original dataset has 9,742 movies rated by 610 users, and a total of 100,836 ratings<br> 
* After removing two insignificant movie genres and filtering out movies with less than 5 ratings, the dataset is reduced to $n_u = 610$ users and $n_m= 3649$ movies. <br> 

In [2]:
def load_data():
       
    movies = pd.read_csv('./ml-latest-small/movie_data.csv')
    users = pd.read_csv('./ml-latest-small/user_data.csv')
    y    = pd.read_csv('./ml-latest-small/y.csv')
    item_vecs = pd.read_csv('./ml-latest-small/item_vecs.csv')

    movie_dict = defaultdict(dict)
    count = 0
    with open('./ml-latest-small/movies.csv', newline='') as movie:
        reader = csv.reader(movie, delimiter=',', quotechar='"')
        for line in reader:
            if count == 0:
                count += 1  # skip header
            else:
                count += 1
                movie_id = int(line[0])
                movie_dict[movie_id]["title"] = line[1]
                movie_dict[movie_id]["genres"] = line[2]
    
    ratings = pd.read_csv('./ml-latest-small/ratings.csv')
    
    return movies, users, y, item_vecs, movie_dict, ratings

In [3]:
movies_df, users_df, y_df, item_vecs_df, movie_dict, ratings_df = load_data()

In [4]:
movies_df.head()

,movieId,year,movie_ave_rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,3.92093,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1995,3.92093,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1995,3.92093,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1995,3.92093,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1995,3.92093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
users_df.head()

,userId,user_ave_rating,user_rating_count,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,4.361233,683,4.318182,4.380952,4.678571,4.536585,4.283951,4.333333,0.0,...,4.297872,5.0,3.470588,4.681818,4.166667,4.307692,4.225,4.12963,4.5,4.285714
1,1,4.361233,683,4.318182,4.380952,4.678571,4.536585,4.283951,4.333333,0.0,...,4.297872,5.0,3.470588,4.681818,4.166667,4.307692,4.225,4.12963,4.5,4.285714
2,1,4.361233,683,4.318182,4.380952,4.678571,4.536585,4.283951,4.333333,0.0,...,4.297872,5.0,3.470588,4.681818,4.166667,4.307692,4.225,4.12963,4.5,4.285714
3,1,4.361233,683,4.318182,4.380952,4.678571,4.536585,4.283951,4.333333,0.0,...,4.297872,5.0,3.470588,4.681818,4.166667,4.307692,4.225,4.12963,4.5,4.285714
4,1,4.361233,683,4.318182,4.380952,4.678571,4.536585,4.283951,4.333333,0.0,...,4.297872,5.0,3.470588,4.681818,4.166667,4.307692,4.225,4.12963,4.5,4.285714


In [6]:
y_df.head()

,rating
0,4.0
1,4.0
2,4.0
3,4.0
4,4.0


In [7]:
item_vecs_df.head()

,movieId,year,movie_ave_rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,3.92093,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1995,3.92093,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1995,3.92093,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1995,3.92093,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1995,3.92093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
print(movies_df.shape)
print(users_df.shape)
print(y_df.shape)
print(item_vecs_df.shape)
print(ratings_df.shape)

(247900, 21)
(247900, 21)
(247900, 1)
(9167, 21)
(100836, 4)


In [10]:
num_user_features = users_df.shape[1] - 3  # remove userid, rating count and ave rating during training
uvs = 3  # user genre vector start
u_s = 3  # Not using first 3 features when training

num_item_features = movies_df.shape[1] - 1  # remove movie id at train time
ivs = 3  # the item genre vector starts at the fourth column(Action, Advanture, ...)
i_s = 1  # the column movie id will not be meaningful for training
scaledata = True  # applies the standard scalar to data if true

In [11]:
# convert the pandas dataframe to numpy
users = users_df.to_numpy()
movies = movies_df.to_numpy()
y = y_df.to_numpy()
item_vecs = item_vecs_df.to_numpy()

In [12]:
# scale training data to improve convergence, z = (x - u) / s
if scaledata:
    # Use StandardScaler to scale the data
    scalerItem = StandardScaler()
    scalerItem.fit(movies)
    movies = scalerItem.transform(movies)

    scalerUser = StandardScaler()
    scalerUser.fit(users)
    users = scalerUser.transform(users)

In [13]:
# split the datasets into traning and testing sets
item_train, item_test = train_test_split(movies, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(users, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training set shape: {item_train.shape}")
print(f"movie/item test  set shape: {item_test.shape}")

movie/item training set shape: (198320, 21)
movie/item test  set shape: (49580, 21)


In [14]:
# Scale the target ratings using a Min Max Scaler to scale the target to be between -1 and 1
scaler = MinMaxScaler((-1, 1))
scaler.fit(y_train.reshape(-1, 1))
ynorm_train = scaler.transform(y_train.reshape(-1, 1))
ynorm_test = scaler.transform(y_test.reshape(-1, 1))
print(ynorm_train.shape, ynorm_test.shape)

(198320, 1) (49580, 1)


##  Create Neural Networks

In [15]:
# Create neural networks for user content and movie content
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_outputs),
])

item_NN = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_outputs),
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# The model output the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 18)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 32)           41888       input_1[0][0]                    
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 32)           42400       input_2[0][0]                    
______________________________________________________________________________________________

2022-08-11 12:58:39.409199: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-11 12:58:39.410334: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [16]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [17]:
tf.random.set_seed(1)
# fit the model with training set (user_train ignore first 3 features, item_train ignore first feature)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], ynorm_train, epochs=25)

Train on 198320 samples
Epoch 1/25
198320/198320 [==============================] - 49s 246us/sample - loss: 0.1351
Epoch 2/25
198320/198320 [==============================] - 49s 245us/sample - loss: 0.1293
Epoch 3/25
198320/198320 [==============================] - 45s 227us/sample - loss: 0.1265
Epoch 4/25
198320/198320 [==============================] - 45s 227us/sample - loss: 0.1246
Epoch 5/25
198320/198320 [==============================] - 45s 227us/sample - loss: 0.1233
Epoch 6/25
198320/198320 [==============================] - 45s 225us/sample - loss: 0.1220
Epoch 7/25
198320/198320 [==============================] - 45s 229us/sample - loss: 0.1209
Epoch 8/25
198320/198320 [==============================] - 45s 228us/sample - loss: 0.1203
Epoch 9/25
198320/198320 [==============================] - 45s 226us/sample - loss: 0.1196
Epoch 10/25
198320/198320 [==============================] - 47s 237us/sample - loss: 0.1190
Epoch 11/25
198320/198320 [============================

In [18]:
# evaluate returns the loss value & metrics values for the model in test mode.
# set verbose means how do you want to 'see' the training progress for each epoch.
# verbose=0 will show you nothing (silent)
# verbose=1 will show you an animated progress bar like this: [==========]
# verbose=2 will just mention the number of epoch like this: Epoch 1/10
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], ynorm_test, verbose=2)

49580/1 - 5s - loss: 0.1075


0.11823970724698178

## Make rating predictions

### New User

In [80]:
# create a new user 
new_user_id = 999
new_rating_ave = 3.0
new_rating_count = 12

new_action = 5.0
new_adventure = 4
new_animation = 1
new_childrens = 1
new_comedy = 5
new_crime = 5
new_documentary = 4
new_drama = 1
new_fantasy = 1
new_film_noir = 5
new_horror = 1
new_musical = 1
new_mystery = 5
new_romance = 1
new_scifi = 5
new_thriller = 1
new_war = 5
new_western = 1
user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave, 
                      new_action, new_adventure, new_animation, new_childrens, 
                      new_comedy, new_crime, new_documentary, new_drama, 
                      new_fantasy, new_film_noir, new_horror, new_musical, 
                      new_mystery, new_romance, new_scifi, new_thriller, 
                      new_war, new_western]])

In [81]:
def predict_uservec(user_vecs, item_vecs, model, u_s, i_s, scaler, ScalerUser, ScalerItem, scaledata=False):
    """ given a user vector, does the prediction on all movies in item_vecs returns
        an array predictions sorted by predicted rating,
        arrays of user and item, sorted by predicted rating sorting index
    """
    # if the vectors need to be scaled, use the fitted StandardScaler() to scale the vectors.
    if scaledata:
        scaled_user_vecs = ScalerUser.transform(user_vecs)
        scaled_item_vecs = ScalerItem.transform(item_vecs)
        y_p = model.predict([scaled_user_vecs[:, u_s:], scaled_item_vecs[:, i_s:]])
    else:
        y_p = model.predict([user_vecs[:, u_s:], item_vecs[:, i_s:]])
    # Scale back the data to the original representation. (0.5 to 5 rating)
    y_pu = scaler.inverse_transform(y_p)
    
    if np.any(y_pu < 0) : 
        print("Error, expected all positive predictions")
    #negate y_pu to get a descending list of indices
    sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  
    sorted_ypu   = y_pu[sorted_index]
    sorted_items = item_vecs[sorted_index]
    sorted_user  = user_vecs[sorted_index]
    return(sorted_index, sorted_ypu, sorted_items, sorted_user)

In [82]:
# np.tile(A, reps), construct an array by repeating A the number of times given by reps.
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = np.tile(user_vec, (len(item_vecs), 1))

# scale the vectors and make predictions for all movies. Return results sorted by rating.
sorted_index, sorted_ypu, sorted_items, sorted_user = predict_uservec(user_vecs, item_vecs, model, u_s, i_s, 
                                                                      scaler, scalerUser, scalerItem, 
                                                                      scaledata=scaledata)

In [83]:
movie_id = sorted_items[:, 0].astype(int)
rating_ave = sorted_items[:, 2].astype(float)
print_pred_movie = defaultdict(list)

In [84]:
for i in range(len(movie_id)):
    print_pred_movie['y_p'].append(sorted_ypu[i, 0])
    print_pred_movie['movie id'].append(movie_id[i])
    print_pred_movie['rating ave'].append(rating_ave[i])
    print_pred_movie['title'].append(movie_dict[movie_id[i]]['title'])
    print_pred_movie['genre'].append(movie_dict[movie_id[i]]['genres'])

In [85]:
pred_new_user_rating = pd.DataFrame(data=print_pred_movie)

In [86]:
pred_new_user_rating.head(10)

,y_p,movie id,rating ave,title,genre
0,4.582815,122886,3.853659,Star Wars: Episode VII - The Force Awakens (2015),Action|Adventure|Fantasy|Sci-Fi|IMAX
1,4.582435,111362,3.833333,X-Men: Days of Future Past (2014),Action|Adventure|Sci-Fi
2,4.581427,116823,3.869565,The Hunger Games: Mockingjay - Part 1 (2014),Adventure|Sci-Fi|Thriller
3,4.580508,122904,3.833333,Deadpool (2016),Action|Adventure|Comedy|Sci-Fi
4,4.580166,122882,3.819149,Mad Max: Fury Road (2015),Action|Adventure|Sci-Fi|Thriller
5,4.579313,187593,3.875000,Deadpool 2 (2018),Action|Comedy|Sci-Fi
6,4.579132,187595,3.900000,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi
7,4.577103,106920,3.920000,Her (2013),Drama|Romance|Sci-Fi
8,4.577035,114935,3.900000,Predestination (2014),Action|Mystery|Sci-Fi|Thriller
9,4.576843,89745,3.869565,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX


The predicted user rating is based on the the user vector which includes a set of user genre rating <br>
For the case that a user only gives a maximum rating for one genre and minimums for the rest, if there's no similar user rating in the user vector, then the predicted rating may not be meaningful.

In [26]:
def get_user_vecs(user_id, user_train, item_vecs, ratings):
    """ given a user_id, return:
        user train/predict matrix to match the size of item_vecs
        y vector with ratings for all rated movies and 0 for others of size item_vecs """

    user_vec_found = False
    for i in range(len(user_train)):
        if user_train[i, 0] == user_id:
            user_vec = user_train[i]
            user_vec_found = True
            break
    if not user_vec_found:
        print("error in get_user_vecs, did not find uid in user_train")
    num_items = len(item_vecs)
    user_vecs = np.tile(user_vec, (num_items, 1))

    y = np.zeros(num_items)
    # walk through movies in item_vecs and get the movies, see if user has rated them
    for i in range(num_items):  
        movie_id_lst = list(ratings.loc[ratings['userId'] == user_id]['movieId'])
        movie_id = item_vecs[i, 0]
        if movie_id in movie_id_lst:
            rating = ratings.loc[ratings['userId'] == user_id].loc[ratings['movieId'] == movie_id].iloc[0]['rating']
        else:
            rating = 0
        y[i] = rating
    return(user_vecs, y)

Note that movies with multiple genre's show up multiple times in the training data. For example,'The Time Machine' has three genre's: Adventure, Action, Sci-Fi

### Exist User

In [62]:
# Predict the rating of user x. Compare the predicted ratings with the model's ratings.
uid = 111

# form a set of user vectors. This is the same vector, transformed and repeated.
user_vecs, y_vecs = get_user_vecs(uid, scalerUser.inverse_transform(user_train), item_vecs, ratings_df)

# scale the vectors and make predictions for all movies. Return results sorted by rating.
sorted_index, sorted_ypu, sorted_items, sorted_user = predict_uservec(user_vecs, item_vecs, model, u_s, i_s, scaler, 
                                                                      scalerUser, scalerItem, scaledata=scaledata)
sorted_y = y_vecs[sorted_index]

In [63]:
movie_id = sorted_items[:, 0].astype(int)
rating_ave = sorted_items[:, 2].astype(float)

In [64]:
print_existing_user = defaultdict(list)

In [65]:
item_features = list(movies_df.columns)

In [66]:
for i in range(len(sorted_ypu)):
    if sorted_y[i] != 0:
        offset = np.where(sorted_items[i, ivs:] == 1)[0][0]
        genre_rating = sorted_user[i, uvs + offset]
        genre = item_features[ivs + offset]

        print_existing_user['y_p'].append(sorted_ypu[i, 0])
        print_existing_user['y'].append(sorted_y[i])
        print_existing_user['user'].append(sorted_user[i, 0].astype(int))
        print_existing_user['user genre ave'].append(genre_rating.astype(float))
        print_existing_user['movie rating ave'].append(rating_ave[i])
        print_existing_user['title'].append(movie_dict[movie_id[i]]['title'])
        print_existing_user['genre'].append(genre)

In [67]:
pred_exist_user_rating = pd.DataFrame(data=print_existing_user)

In [68]:
pred_exist_user_rating.head(15)

,y_p,y,user,user genre ave,movie rating ave,title,genre
0,3.985988,3.5,111,3.260000,4.232394,"Princess Bride, The (1987)",Romance
1,3.985378,4.5,111,3.326613,4.429022,"Shawshank Redemption, The (1994)",Drama
2,3.933220,5.0,111,5.000000,3.916667,Jackass 3D (2010),Documentary
3,3.917574,5.0,111,3.260000,4.164134,Forrest Gump (1994),Romance
4,3.887069,5.0,111,3.382948,3.986111,Kingsman: The Secret Service (2015),Comedy
5,3.876984,4.0,111,3.382948,4.000000,"Secret Life of Walter Mitty, The (2013)",Comedy
6,3.865362,2.5,111,3.382948,3.890625,Zootopia (2016),Comedy
7,3.850085,5.0,111,3.382948,3.833333,Deadpool (2016),Comedy
8,3.850075,4.5,111,3.382948,3.800000,The Boss Baby (2017),Comedy
9,3.846318,5.0,111,3.382948,3.916667,"Wolf of Wall Street, The (2013)",Comedy


### Find Similar Movies

A similarity measure is the squared distance between the two vectors $ \mathbf{v_m^{(k)}}$ and $\mathbf{v_m^{(i)}}$ :
$$\left\Vert \mathbf{v_m^{(k)}} - \mathbf{v_m^{(i)}}  \right\Vert^2 = \sum_{l=1}^{n}(v_{m_l}^{(k)} - v_{m_l}^{(i)})^2$$

In [69]:
def sq_dist(a,b):
    """
    Returns the squared distance between two vectors
    """
    
    d = np.sum(np.square(a - b), axis = 0)
    
    return (d)

A matrix of distances between movies can be computed once when the model is trained and then reused for new recommendations without retraining.<br>
We can build a model to run the movie vectors to generate the movie feature vector $v_m$ for each of the movies.<br>

In [70]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))    
vm_m = item_NN(input_item_m)                                       
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                     
model_m = Model(input_item_m, vm_m)                                
model_m.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 32)           42400       input_3[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_l2_normalize_2/Squa [(None, 32)]         0           sequential_1[1][0]               
__________________________________________________________________________________________________
tf_op_layer_l2_normalize_2/Sum  [(None, 1)]          0           tf_op_layer_l2_normalize_2/Square
____________________________________________________________________________________________

The item_vecs must be scaled to use with the trained model. The result of the prediction is a 32 entry feature vector for each movie.

In [74]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print("Shape of movie feature vectors is {}".format(vms.shape))

Shape of movie feature vectors is (9167, 32)


The dataset contains 3649 unique movies, but same movie will appear as a separate vector for each of its genres.

In [75]:
def get_item_genre(item, ivs, item_features):
    # np.where(item[ivs:] == 1) will return (array([x]),) where x is the index of the column equal to 1
    offset = np.where(item[ivs:] == 1)[0][0]
    genre = item_features[ivs + offset]
    return(genre, offset)

In [76]:
dim = len(vms)
dist = np.zeros((dim,dim))

# create the matrix
for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])

# The diagonal contains all the products of the same movie, so mask the diagonal to avoid selecting the same movie
m_dist = np.ma.masked_array(dist, mask=np.identity(dist.shape[0]))  

In [77]:
table = defaultdict(list)
for i in range(50):
    min_idx = np.argmin(m_dist[i])
    movie1_id = int(item_vecs[i,0])
    movie2_id = int(item_vecs[min_idx,0])
    genre1,_  = get_item_genre(item_vecs[i,:], ivs, item_features)
    genre2,_  = get_item_genre(item_vecs[min_idx,:], ivs, item_features)
    
    table["movie1"].append(movie_dict[movie1_id]['title'])
    table["genre1"].append(genre1)
    table["movie2"].append(movie_dict[movie2_id]['title'])
    table["genre2"].append(genre2)

In [78]:
similar_movie = pd.DataFrame(data=table)

In [79]:
similar_movie

,movie1,genre1,movie2,genre2
0,Toy Story (1995),Adventure,"Lion King, The (1994)",Adventure
1,Toy Story (1995),Animation,Wallace & Gromit: A Close Shave (1995),Animation
2,Toy Story (1995),Children,"Little Princess, A (1995)",Children
3,Toy Story (1995),Comedy,Wallace & Gromit: A Close Shave (1995),Comedy
4,Toy Story (1995),Fantasy,Groundhog Day (1993),Fantasy
5,Jumanji (1995),Adventure,"War, The (1994)",Adventure
6,Jumanji (1995),Children,"Sandlot, The (1993)",Children
7,Jumanji (1995),Fantasy,"Prophecy, The (1995)",Fantasy
8,Grumpier Old Men (1995),Comedy,Bad Boys (1995),Comedy
9,Grumpier Old Men (1995),Romance,"Bridges of Madison County, The (1995)",Romance
